In [1]:
import numpy as np
from pandas import read_csv
posts_data = read_csv("./risk/posts_data.csv")

In [2]:
from nltk.corpus import stopwords
badwords = [
u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок'
] + stopwords.words('russian')

In [3]:
def files(filenames):
    for name in filenames:
        with open(path + name, 'r', encoding='utf-8') as f:
            yield f.read()

In [4]:
from os import listdir
path = '../posts_utf8/'
filenames = listdir(path)

In [5]:
# избавляемся от текстов без тегов активности
from math import isnan
y_dict = {k: v for k, v in zip(posts_data['ID'], posts_data['tag_activity']) if isinstance(v, str) and k in filenames}

In [6]:
print(len(y_dict))

30502


In [7]:
# не учитываем тексты длиной менее 100 символов, так как с большой вероятностью они не информативны
# и представляют собой подписи к фото/видео
names = [name for name in y_dict.keys() if len(open(path + name, 'r', encoding='utf-8').read()) > 100]
print(len(names))

23751


#### TF-IDF + Logistic regression

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), analyzer='word', max_features=20000, stop_words=badwords)
vectorizer.fit(files(names))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['я', 'а', 'да', 'но', 'тебе', 'мне', 'ты', 'и', 'у', 'на', 'ща', 'ага', 'так', 'там', 'какие', 'который', 'какая', 'туда', 'давай', 'короче', 'кажется', 'вообще', 'ну', 'не', 'чет', 'неа', 'свои', 'наше', 'хотя', 'такое', 'например', 'кароч', 'как-то', 'нам', 'хм', 'всем', 'нет', 'да', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
X = vectorizer.transform(files(names))
y = [y_dict[k] for k in names]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

logreg = LogisticRegression(solver='newton-cg',multi_class='multinomial')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = logreg.fit(X_train, y_train)

In [11]:
model.score(X_test, y_test)

0.7046937486844875

In [14]:
len(activities)

19

In [12]:
activities = set(y_test)
preds = model.predict(X_test)
for activity in activities:
    print(activity, '\t', sum([x == activity for x in preds]), sum([x == activity for x in y_test]))

Спелеология 	 5 23
Ски-тур 	 31 87
Мультигонки 	 93 130
Вело 	 6 61
Снегоступинг 	 2 16
Вода 	 43 98
Путешествия 	 175 271
Слэклайн 	 0 1
Бэккантри/Фрирайд 	 172 219
Альпинизм 	 3199 2253
Скайраннинг 	 36 77
Воздух 	 58 119
Туризм 	 1 72
Ледолазание/drytoolling 	 73 123
Скалолазание 	 621 719
BASE 	 2 32
Горный туризм 	 158 288
Горные лыжи/Сноуборд 	 76 152
Ropejumping 	 0 10


In [17]:
from sklearn.metrics import classification_report

print( classification_report(y_test, preds) )

                         precision    recall  f1-score   support

                   BASE       0.00      0.00      0.00        32
            Ropejumping       0.00      0.00      0.00        10
              Альпинизм       0.68      0.96      0.79      2253
      Бэккантри/Фрирайд       0.72      0.56      0.63       219
                   Вело       0.83      0.08      0.15        61
                   Вода       0.72      0.32      0.44        98
                 Воздух       0.83      0.40      0.54       119
   Горные лыжи/Сноуборд       0.62      0.31      0.41       152
          Горный туризм       0.59      0.32      0.42       288
Ледолазание/drytoolling       0.79      0.47      0.59       123
            Мультигонки       0.78      0.56      0.65       130
            Путешествия       0.55      0.35      0.43       271
            Скайраннинг       0.69      0.32      0.44        77
           Скалолазание       0.90      0.77      0.83       719
                Ски-тур 

c:\users\julia-win\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
